In [11]:
from helpers.idf_helpers import *
from helpers.plot_helpers import *
from studies import run_study

import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, SelectMultiple
from eppy.modeleditor import IDF
from IPython.display import display

import os

In [12]:
outputs = pd.read_excel('Output Variables.xlsx')

In [13]:
temp = outputs[outputs['Units']=='C']

In [14]:
@interact
def get_output(output = temp['Output Name']):
    return temp[temp['Output Name']==output]

interactive(children=(Dropdown(description='output', options=('Site Outdoor Air Drybulb Temperature', 'Site Ou…

In [15]:
var_list = ['Zone Mean Radiant Temperature',
            'Zone Mean Air Temperature',
            'Zone Operative Temperature',
            'Zone Mean Air Dewpoint Temperature',
            'Zone Outdoor Air Wetbulb Temperature', 
            'Zone Air Temperature',
            'Zone Thermostat Air Temperature',
            'Zone Thermostat Heating Setpoint Temperature',
            'Zone Thermostat Cooling Setpoint Temperature',
      '     Zone Adaptive Comfort Operative Temperature Set Point',
           'Zone Heat Index']

In [16]:
idffile = '/home/paula/Documentos/Doctorado/Desarrollo/eppyProject/input/simple_room_1_window_1_door_unitary_System_sch_constant.idf'
iddfile = '/usr/local/EnergyPlus-9-4-0/Energy+.idd'
epwfile = '/home/paula/Documentos/Doctorado/Desarrollo/eppyProject/input/Wheather file/FRA_Paris.Orly.071490_IWEC.epw'
idf2 = initialization(idffile, iddfile, epwfile)

In [17]:
@interact
def show_not_null_classes(classes = notnullobj(idf2)):
    return epobject_to_df(idf2.idfobjects[classes])

interactive(children=(Dropdown(description='classes', options=('VERSION', 'SIMULATIONCONTROL', 'BUILDING', 'TI…

#### Simulationontrol:
Do_Zone_Sizing_Calculation Necesita del objejo Sizing:Zone para funcionar


Do_System_Sizing_Calculation Necesita del objejo Sizing:Sytem para funcionar


#### Building:

...No tengo nada específicado. Quizás debería poner algo más..o no?? xq contra menos variables tenga quizás mejor?






In [18]:
idf = idf2
objs_to_delete=[]
scriptname='variable_study'
vars_name=var_list
objs_to_delete = ['PEOPLE', 'LIGHTS', 'ELECTRICEQUIPMENT']

## tengo que hacer cambios para que esta función también funcione desde jupyter notebook

for obj in objs_to_delete:
    for o in idf.idfobjects[obj]:
        idf.removeidfobject(o)
# TODO hacer esto general
# hacer un remove de las variables que ya estaban

for n in range(len(vars_name)):
    idf.newidfobject('OUTPUT:VARIABLE')
    idf.idfobjects['OUTPUT:VARIABLE'][n].Variable_Name = vars_name[n]
# if idf.idfobjects['HVACTemplate:Thermostat']:
#     schedule_name = idf.idfobjects['HVACTemplate:Thermostat'][0].Heating_Setpoint_Schedule_Name
#     idf.newidfobject('OUTPUT:VARIABLE')
#     idf.idfobjects['OUTPUT:VARIABLE'][-1].Key_Value = schedule_name
#     idf.idfobjects['OUTPUT:VARIABLE'][-1].Variable_Name = 'Schedule Value'

# newdir = os.path.basename(__file__).replace('.py','')


foldername = scriptname.replace('.py', '')

try:
    os.mkdir(foldername)
except FileExistsError:
    print("Directory already exists")
abs_newdir = '/home/paula/Documentos/Doctorado/Desarrollo/eppyProject/variable_study'
#idf.saveas(abs_newdir + '/idffile.idf')
idf.run(output_directory=abs_newdir)

Directory already exists

/usr/local/EnergyPlus-9-4-0/energyplus --expandobjects --idd /usr/local/EnergyPlus-9-4-0/Energy+.idd --output-directory /home/paula/Documentos/Doctorado/Desarrollo/eppyProject/variable_study --weather /home/paula/Documentos/Doctorado/Desarrollo/eppyProject/input/Wheather file/FRA_Paris.Orly.071490_IWEC.epw /home/paula/Documentos/Doctorado/Desarrollo/eppyProject/in.idf



EnergyPlusRunError: 

Contents of EnergyPlus error file at /home/paula/Documentos/Doctorado/Desarrollo/eppyProject/variable_study/eplusout.err
Program Version,EnergyPlus, Version 9.4.0-998c4b761e, YMD=2021.02.23 19:51,
   ************* Beginning Zone Sizing Calculations
   ** Warning ** ProcessScheduleInput: Schedule:Constant="ALWAYS OFF DISCRETE", Blank Schedule Type Limits Name input -- will not be validated.
   ** Warning ** ProcessScheduleInput: Schedule:Constant="ALWAYS ON CONTINUOUS", Blank Schedule Type Limits Name input -- will not be validated.
   ** Warning ** GetHTSurfaceData: Surfaces with interface to Ground found but no "Ground Temperatures" were input.
   **   ~~~   ** Found first in surface=SURFACE 1
   **   ~~~   ** Defaults, constant throughout the year of (18.0) will be used.
   ** Warning ** CheckUsedConstructions: There are 21 nominally unused constructions in input.
   **   ~~~   ** For explicit details on each unused construction, use Output:Diagnostics,Displa

In [ ]:
%%bash
cp studies/ReadVarsESO variable_study/


In [ ]:
%%bash
ls variable_study/

In [ ]:
%%bash
./variable_study/ReadVarsESO

In [ ]:
data = eso_to_ts('variable_study/eplusout.csv')

In [ ]:
winterperiod = ['2000-01-08','2000-01-10']
summerperiod= ['2000-08-08','2000-08-10']

In [ ]:
datawinter = data[winterperiod[0]:winterperiod[1]]

In [ ]:
datawinter.plot(title = 'invierno',figsize = (12,5),grid =True).legend(loc = 'upper left',bbox_to_anchor=(1.05, 1))

In [ ]:
winter[[]]